In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from os import listdir, makedirs
from os.path import join, isfile, isdir, exists
from datetime import datetime
import pandas as pd
import numpy as np
import math
from tqdm import tqdm

In [3]:
# Input
root_folder = '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe'
ActiPAL_15s_epoch_proc_folder = '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/Data/processed_data/ActiPAL/epoch_data/15s_ref'
ActiPAL_20hz_raw_folder = '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe/activPAL_raw'

# sleep time input_data
sleep_time_file = '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/Data/metadata/record_availability_final_pal.csv'

# Output
ActiPAL_output_folder = '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/processed data/thigh_minute_data'

### Load Sleep Data

In [4]:
raw_files = [f for f in listdir(ActiPAL_20hz_raw_folder) if isfile(join(ActiPAL_20hz_raw_folder, f))]
epoch_files = [f for f in listdir(ActiPAL_15s_epoch_proc_folder) if isfile(join(ActiPAL_15s_epoch_proc_folder, f))]

data_dict = {f.split('-')[0]: {'epoch': join(ActiPAL_15s_epoch_proc_folder, f)} for f in epoch_files}
print(len(data_dict))

# print(len(epoch_files))
# for ef in epoch_files:
#   key = ef.split('-')[0]
#   if key in data_dict.keys():
#     data_dict[key]['epoch'] = join(ActiPAL_15s_epoch_proc_folder, ef)

165


In [5]:
print(data_dict['HOA1'])

{'epoch': '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/Data/processed_data/ActiPAL/epoch_data/15s_ref/HOA1-AP479873 06May19 03-00pm for 6d 17h 49m-CREA-PB08100943-15sEpochs_15sREF.csv'}


In [6]:
sleep_df = pd.read_csv(sleep_time_file)
sleep_df.head(2)

,ID,Participant,activpal_Start_time,activpal_End_time,activpal_Time_difference,actigraph_Start_time,actigraph_End_time,actigraph_Time_difference,Overlap,age_yr,sex,BMI,bodymass_kg,AG_data,AP_data,"Side_TKA (1=L,2=R)",slnumnew,nonwearmins,sleepmins,nonwearpct,justnonwear,justsleep,begin_time_pal,end_time_pal
0,0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d,160h 59m 59d,52.0,female,76.86261,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-06 14:59:46,2019-05-06 22:00:00
1,0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d,160h 59m 59d,52.0,female,76.86261,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-07 06:00:00,2019-05-07 22:00:00


### Generate statistical features

In [ ]:
# FREQUENCY = 20 #HZ amount of the accelerometer
# REFERENCE_EPOCH = 15 #Epoch value that EE is given
# TIME_EPOCH_DICT = {
#     # 'Epoch1': FREQUENCY * 1,
#     # 'Epoch5': FREQUENCY * 5,
#     'Epoch30': FREQUENCY * 30,
#     # 'Epoch60': FREQUENCY * 60
# }

In [9]:
def met_to_intensity_ground(row):
  ee = complete_df['per_minute_means'][row.name]
  return 1 if ee <= 1.5 else 2 if ee < 3 else 3



def get_bmi_range(row):
  bmi = complete_df['BMI'][row.name]
  if(18.5<=bmi<25):return 'normal'
  elif(25<=bmi<30):return 'overweight'
  elif(30<=bmi):return 'obese'
  else:'error'

In [ ]:
s = ['A']*10
print(s)

['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A']


In [10]:

complete_df = pd.DataFrame()
output_file_name = join(ActiPAL_output_folder, 'per_minute_EE_means_thigh_reference.csv')
for _, row in tqdm(sleep_df.iterrows(), total=sleep_df.shape[0]):

  user_id = row['Participant']
  sex = row['sex']
  bmi = row['BMI']
  # if(user_id!='HOA1'):
  #   continue
  begin_time = row['begin_time_pal']
  end_time = row['end_time_pal']
  # print(user_id,begin_time,end_time)
  
  temp_df = pd.DataFrame()
  # get respective epoch dataset
  try:
    epoch_df = pd.read_csv(data_dict[user_id]['epoch'])
    epoch_df = epoch_df[(begin_time < epoch_df['Time_stamp']) & (epoch_df['Time_stamp'] < end_time)]
    # print("df len",len(epoch_df))
    epoch_df_grouped = epoch_df.groupby(epoch_df.index // 4)
    per_minute_means = epoch_df_grouped['thigh_EE'].mean()
    temp_df['per_minute_means'] = per_minute_means
    temp_df['BMI'] = [bmi]*len(per_minute_means)
    temp_df['sex'] = [sex]*len(per_minute_means)
    temp_df.insert(0,'Participant',[user_id]*len(per_minute_means))
    # temp_df['Participant'] = [user_id]*len(hourly_means)
    # prtemp_lenint('',len(temp_df))
    complete_df = pd.concat([complete_df,temp_df])
    # print(len(epoch_df))
    # print(len(epoch_df_grouped))
  except KeyError:
    continue
print('comp',len(complete_df))
complete_df.reset_index(drop=True, inplace=True)
complete_df['Category'] = complete_df.apply(met_to_intensity_ground,axis=1)
complete_df['BMI_range'] = complete_df.apply(get_bmi_range,axis=1)
print("completed")
complete_df.to_csv(output_file_name)



  


100%|██████████| 1367/1367 [01:27<00:00, 15.59it/s]


comp 801578
completed


In [11]:
complete_df = pd.read_csv('/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/processed data/thigh_minute_data/per_minute_EE_means_thigh_reference.csv')
grouped_df = complete_df.groupby(['Participant','sex','BMI_range', 'Category']).size().unstack(fill_value=0)
output_g_file_name = join(ActiPAL_output_folder, 'per_minute_EE_means_thigh_referece_grouped.csv')
grouped_df.to_csv(output_g_file_name)
# for _, row in tqdm(complete_df.iterrows(), total=complete_df.shape[0]):
#   print(row)

In [13]:
all_df = pd.read_csv('/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/processed data/thigh_minute_data/per_minute_EE_means_thigh_referece_grouped.csv')

female = all_df['sex'] == 'female'
male = all_df['sex'] == 'male'
obese = all_df['BMI_range'] == 'obese'
overweight = all_df['BMI_range'] == 'overweight'
normal = all_df['BMI_range'] == 'normal'


male_df = all_df[male]
female_df = all_df[female]
obese_df = all_df[obese]
overweight_df = all_df[overweight]
normal_df = all_df[normal]

# print(['all,male,female',len(all_df),len(male_df),len(female_df)])
# print(['all,male,female',all_df['1'].mean,len(male_df),len(female_df)])

with pd.ExcelWriter(join(ActiPAL_output_folder,'thigh_reference_minute_values.xlsx')) as writer: 
    all_df.to_excel(writer, sheet_name='all')
    female_df.to_excel(writer, sheet_name='female')
    male_df.to_excel(writer, sheet_name='male')
    obese_df.to_excel(writer, sheet_name='obese')
    overweight_df.to_excel(writer, sheet_name='overweight')
    normal_df.to_excel(writer, sheet_name='normal')

In [ ]:
data = pd.DataFrame()
data['X'] = [1,2,3,4,5,6,7,8,9]
data['Y'] = [0,2,4,6,8,10,12,14,16]
data['Z'] = [3,6,9,12,15,18,21,24,27]

grouped = data.groupby(data.index // 2)
new_d = data.iloc[::2]
new_d['X_sum'] = grouped['X'].sum()
print(data)
# print(grouped)
print(new_d)
# print(mean_g)

   X   Y   Z
0  1   0   3
1  2   2   6
2  3   4   9
3  4   6  12
4  5   8  15
5  6  10  18
6  7  12  21
7  8  14  24
8  9  16  27
   X   Y   Z  X_sum
0  1   0   3    3.0
2  3   4   9   11.0
4  5   8  15    9.0
6  7  12  21    NaN
8  9  16  27    NaN


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
